In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from subprocess import run
import re

In [2]:
# set up filenames and paths
file = "SealedCore"
nd_directory = "~/Documents/Enceladus/1D_files/output"
bm_directory =  "~/Documents/Enceladus/1D_files"

nd_filename = f"{nd_directory}/{file}.nd"
bm_filename = f"{bm_directory}/{file}.bm"

In [3]:
# read in the input .bm file
def Read_bm_mod(file_name):
    print(file_name)
    with open(file_name, 'r') as data:
        rad = []
        vp = []
        vs = []
        rho = []
        layernames = []
        bounds = []
        i = -1
        j = 0
        k = 0
        for line in data:
            if 'UNITS' in line: # check scaling units
                units = line.split()
                units = units[1]
                
            if j == 1: # wait until end of header (COLUMNS) before starting model data section
                i = i+1
                if line == "\n": # skip whitespace
                    i = i-1
                if '#' in line: # if this is a layer name line, add it to the list of layers
                    print(line)
                    layernames.append(line.replace('\n',''))
                    bounds.append(i)
                try: # i.e. execute this block if this is a line containing data
                    p = line.split()
                    float(p[1])
                    rad.append(float(p[radp]))
                    vp.append(float(p[vpp]))
                    vs.append(float(p[vsp]))
                    rho.append(float(p[rhop]))
                except :
                    pass
                
            if 'COLUMNS' in line:
                p = line.split()
                p.pop(0)
                rhop = p.index('rho')
                radp = p.index('radius')
                vpp = p.index('vp')
                vsp = p.index('vs')
                j = 1 # signals end of header
        
        r = max(rad) # obtain radius
            
    return rad, vp, vs, rho, bounds, r, layernames, units

In [4]:
dep, vp, vs, rho, bounds, rmax, layernames, units = Read_bm_mod(bm_filename)

C:/Users/mercu/Downloads/1D_files/CadekEPSCBrantutW2.bm
0
0
# regolith

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
# outer-core

20
21
22
# inner-core

23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
[252100.0, 251692.0, 250467.0, 249243.0, 248018.0, 246794.0, 245569.0, 244345.0, 243120.0, 241896.0, 240671.0, 239447.0, 238222.0, 236998.0, 235773.0, 234549.0, 233324.0, 232100.0, 232100.0, 182500.0, 182500.0, 182450.0, 173548.0, 164414.0, 155279.0, 146145.0, 137011.0, 127877.0, 118743.0, 109609.0, 100475.0, 91341.0, 82207.0, 73073.0, 63939.0, 54805.0, 45670.0, 36536.0, 27402.0, 18268.0, 9134.0, 0.0]
['# regolith', '# outer-core', '# inner-core']
252100.0
[0, 19, 22]
m


In [5]:
# function to write .nd model
def Make_nd_mod(dep, vp, vs, rho, bounds, rmax, layernames, units, output_filename):
    j = 0
    k = 0
    lines = []
    layers = []
    # sort out scaling units
    if units =='m':
        m = 1
    elif units =='km':
        m = 0   
    for layername in layernames: # reformat layer names
        layername = layername.replace('# ', '')
        layername = layername.replace(' ', '-')
        layers.append(layername)
    for i in range(0,(len(dep)+len(layernames))):
        if i in bounds:
            if layers[j] != 'regolith': # avoid writing the regolith layer name for TauP compatibility
                lines.append(f"{layers[j]}")
            j = j+1
        elif dep[k] == dep[k-1] and i-1 not in bounds: # if there is a repeat depth entry and the layer has not been named
            layers.insert(j,f"unnamed-layer-{j}")
            lines.append(f"unnamed-layer-{j}")
            lines.append(f"{round((rmax - dep[k])/1000**m, 2)}   {vp[k]/1000**m}   {vs[k]/1000**m}   {rho[k]/1000**m}")
            k = k+1
            j = j+1
        else :
            lines.append(f"{round((rmax - dep[k])/1000**m, 2)}   {vp[k]/1000**m}   {vs[k]/1000**m}   {rho[k]/1000**m}")
            k = k+1
    if '# mantle' not in layernames: # add arbitrary mantle layer name if it was missing from .bm
        lines.insert(1,f"{lines[1].split()[0]}   {lines[0].split()[1]}   {lines[0].split()[2]}   {lines[0].split()[3]}")
        lines.insert(2,'mantle')
    for line in lines:
        print(line)

    with open(output_filename, 'w') as fout:
        for line in lines:
            fout.write("%s\n" % line)
    
    return file, layers

In [6]:
file, layers = Make_nd_mod(dep,vp,vs,rho,bounds,rmax,layernames,units,nd_filename)

0
1
2
0.0   4.2504   2.1526   0.93
0.41   4.2504   2.1526   0.93
mantle
0.41   4.2418000000000005   2.1481   0.93
1.63   4.2158999999999995   2.1346   0.93
2.86   4.1897   2.121   0.93
4.08   4.163399999999999   2.1071999999999997   0.93
5.31   4.136699999999999   2.0933   0.93
6.53   4.1097   2.0791999999999997   0.93
7.75   4.0825   2.065   0.93
8.98   4.055   2.0507   0.93
10.2   4.0273   2.0362   0.93
11.43   3.9991999999999996   2.0216   0.93
12.65   3.9709   2.0068   0.93
13.88   3.9421999999999997   1.9918   0.93
15.1   3.9134   1.9768   0.93
16.33   3.8841   1.9615   0.93
17.55   3.8546   1.9461   0.93
18.78   3.8248   1.9305   0.93
20.0   3.7946   1.9148   0.93
outer-core
20.0   1.4   0.0   1.03
69.6   1.4   0.0   1.03
inner-core
69.6   2.5431999999999997   1.499   2.29
69.65   2.5431999999999997   1.499   2.29
78.55   2.6688   1.573   2.36
87.69   2.7989   1.649   2.43
96.82   2.9295   1.724   2.5
105.95   3.0593000000000004   1.799   2.55
115.09   3.187   1.873   2.6
124.22 